In [1]:
import torch
print(torch.cuda.is_available())  # sollte True sein
print(torch.cuda.get_device_name(0))  # sollte deine H100 anzeigen


True
NVIDIA H100 NVL


In [3]:
# Vollständige Minimalversion eines Trainingsskripts für ein SimilarBERT-Modell mit Triplet Loss, basierend auf textbasierten Triplets in einer CSV-Datei mit den Spalten anchor, positive, negative.
# pip install -U sentence-transformers # Diesen Befehl im bash ausführen!

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from sentence_transformers import SentenceTransformer


In [3]:
# Trainingsskript: train_similarbert.py
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pandas as pd
import os


In [4]:
# ======================
# 1. Daten laden
# ======================
csv_path = 'text_triplets.csv'  # Pfad zu deiner CSV mit anchor, positive, negative
df = pd.read_csv(csv_path)

# Liste von InputExamples erzeugen
train_examples = [
    InputExample(texts=[row['anchor'], row['positive'], row['negative']])
    for _, row in df.iterrows()
]



In [5]:
# ======================
# 2. Modell & Loss definieren
# ======================
model = SentenceTransformer('intfloat/multilingual-e5-large')  # oder z.B. 'all-MiniLM-L6-v2'

# TripletLoss-Objekt erstellen
train_loss = losses.TripletLoss(model=model)

# DataLoader mit Trainingsbeispielen
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)



In [6]:
# ======================
# 3. Training
# ======================
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    warmup_steps=100,
    show_progress_bar=True,
    output_path='multilingual-e5-large-EdPub',           # ← finales Modell am Schluss
    checkpoint_path='checkpoints/',            # ← Teilschritte hier speichern
    checkpoint_save_steps=500,                 # ← alle 500 Schritte speichern
    checkpoint_save_total_limit=3              # ← max. 3 Checkpoints behalten
)
print("✅ Training abgeschlossen! Modell gespeichert in 'multilingual-e5-large-EdPub'")


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,4.556800
1000,4.254600


✅ Training abgeschlossen! Modell gespeichert in 'multilingual-e5-large-EdPub'


In [ ]:
#Modell später neu laden:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multilingual-e5-large-EdPub')


In [ ]:
#Modell verwenden wie jedes andere SBERT-Modell:
emb = model.encode("Ein Beispielsatz", convert_to_tensor=True)
